[blog.tensorflow.org](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)

[Keras Tuner](https://github.com/keras-team/keras-tuner) is an easy-to-use, distributable hyperparameter optimization framework that solves the pain points of performing a hyperparameter search.
Keras Tuner makes it easy to define a search space and leverage included algorithms to find the best hyperparameter values.

Keras Tuner comes with Bayesian Optimization, Hyperband, and Random Search algorithms built-in, and is also designed to be easy for researchers to extend in order to experiment with new search algorithms.

Here’s a simple end-to-end example. First, we define a model-building function. It takes an hp argument from which you can sample hyperparameters, such as hp.Int('units', min_value=32, max_value=512, step=32) (an integer from a certain range). Notice how the hyperparameters can be defined inline with the model-building code. The example below creates a simple tunable model that we’ll train on CIFAR-10

In [1]:
import tensorflow as tf

# takes an hp argument from which you can sample hyperparameters,
# such as hp.Int('units', min_value=32, max_value=512, step=32) (an integer from a certain range)
def build_model(hp):

    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = inputs

    for i in range(hp.Int('conv_blocks', 3, 5, default=3)):
        filters = hp.Int('filters_' + str(i), 32, 256, step=32)

        for _ in range(2):
            x = tf.keras.layers.Convolution2D(
                filters, kernel_size=(3, 3), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.ReLU()(x)

        if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
            x = tf.keras.layers.MaxPool2D()(x)
        else:
            x = tf.keras.layers.AvgPool2D()(x)

    x = tf.keras.layers.GlobalAvgPool2D()(x)
    x = tf.keras.layers.Dense(
        hp.Int('hidden_size', 30, 100, step=10, default=50),
        activation='relu')(x)
    x = tf.keras.layers.Dropout(
        hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(x)

    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    return model

2023-05-09 10:45:01.482564: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 10:45:02.137491: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Next, instantiate a tuner. You should specify the model-building function, and the name of the objective to optimize (whether to minimize or maximize is automatically inferred for built-in metrics -- for custom metrics you can specify this via the kerastuner.Objective class). In this example, Keras tuner will use the Hyperband algorithm for the hyperparameter search

In [11]:
import keras_tuner as kt
#import kerastuner as kt

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


Next we’ll download the CIFAR-10 dataset using [TensorFlow Datasets](https://www.tensorflow.org/datasets), and then begin the hyperparameter search. To start the search, call the search method. This method has the same signature as 'keras.Model.fit'

In [3]:
%%time

import tensorflow_datasets as tfds

data = tfds.load('cifar10')
train_ds, test_ds = data['train'], data['test']

def standardize_record(record):
    return tf.cast(record['image'], tf.float32) / 255., record['label']

train_ds = train_ds.map(standardize_record).cache().batch(64).shuffle(10000)
test_ds = test_ds.map(standardize_record).cache().batch(64)

tuner.search(train_ds,
             validation_data=test_ds,
             epochs=30,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])

2023-05-09 10:45:05.222711: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-09 10:45:05.257853: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-09 10:45:05.257944: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-09 10:45:05.260166: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-09 10:45:05.260236: I tensorflow/compile

Cause: Unable to locate the source code of <function standardize_record at 0x7f6ee98bfe50>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


r/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-09 10:45:06.002247: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-09 10:45:06.002366: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-09 10:45:06.002379: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1722] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2023-05-09 10:45:06.002425: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to r

Cause: Unable to locate the source code of <function standardize_record at 0x7f6ee98bfe50>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


CPU times: user 768 ms, sys: 312 ms, total: 1.08 s
Wall time: 1.63 s


Each model will train for at most 30 epochs, and two iterations of the Hyperband algorithm will be run. Afterwards, you can retrieve the best models found during the search by using the get_best_models function

In [4]:
%%time

best_model = tuner.get_best_models(1)[0]

# You can also view the optimal hyperparameter values found by the search:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
best_hyperparameters

CPU times: user 462 ms, sys: 57.1 ms, total: 520 ms
Wall time: 620 ms


# Built-in Tunable Models
In addition to allowing you to define your own tunable models, Keras Tuner provides two built-in tunable models: HyperResnet and HyperXception.

These models search over various permutations of the ResNet and Xception architectures, respectively.
These models can be used with a Tuner like this:

In [5]:
# Please install scikit-learn (sklearn) before using the `BayesianOptimization` with `pip install keras-tuner[bayesian]
!pip install keras-tuner[bayesian]

In [8]:
%%time

tuner = kt.tuners.BayesianOptimization(
    kt.applications.HyperResNet(input_shape=(256, 256, 3), classes=10),
    objective='val_accuracy',
    max_trials=50,
    project_name='untitled2'
)

CPU times: user 1.1 s, sys: 46.2 ms, total: 1.15 s
Wall time: 1.21 s


# Additional

## Distributed Tuning
With Keras Tuner, you can do both data-parallel and trial-parallel distribution. That is, you can use tf.distribute.Strategy to run each Model on multiple GPUs, and you can also search over multiple different hyperparameter combinations in parallel on different workers.

No code changes are needed to perform a trial-parallel search. Simply set the KERASTUNER_TUNER_ID, KERASTUNER_ORACLE_IP, and KERASTUNER_ORACLE_PORT environment variables, for example as shown in the bash script here:

In [10]:
!export KERASTUNER_TUNER_ID="chief"
!export KERASTUNER_ORACLE_IP="127.0.0.1"
!export KERASTUNER_ORACLE_PORT="8000"
!python run_my_search.py

python: can't open file '/mnt/d/gits/forecasting/TestProjects/Keras Tuner CIFAR-10 multiclasses/run_my_search.py': [Errno 2] No such file or directory


+The tuners coordinate their search via a central Oracle service that tells each tuner which hyperparameter values to try next.
For more information, see our [Distributed Tuning guide](https://keras-team.github.io/keras-tuner/tutorials/distributed-tuning/).

## Custom Training Loops

The `kerastuner.Tuner` class can be subclassed to support advanced uses such as:
* Custom training loops (GANs, reinforcement learning, etc.)
* Adding hyperparameters outside of the model building function (preprocessing, data augmentation, test time augmentation, etc.)

For more information, see our [Tuner Subclassing guide](https://keras-team.github.io/keras-tuner/tutorials/subclass-tuner/).

Here’s a simple example:

## Tuning Scikit-learn Models
Despite its name, Keras Tuner can be used to tune a wide variety of machine learning models. In addition to built-in Tuners for Keras models, Keras Tuner provides a built-in Tuner that works with Scikit-learn models. Here’s a simple example of how to use this tuner:

For more information on Keras Tuner, please see the [Keras Tuner website](https://keras-team.github.io/keras-tuner/) or [the Keras Tuner GitHub](https://github.com/keras-team/keras-tuner). Keras Tuner is an open-source project developed entirely on GitHub. If there are features you’d like to see in Keras Tuner, please open a GitHub issue with a feature request, and if you’re interested in contributing, please take a look at our [contribution guidelines](https://keras-team.github.io/keras-tuner/contributing/).
